# Tensorflow(using Pyfunc for model)

In [1]:
import tensorflow as tf

# data import
import scipy.io
ratdata = scipy.io.loadmat('chrono_B069_rawdata.mat')

In [2]:
# trial index starts from 0 

def trialdata(ratdata,trial):
    if ratdata['rawdata']['pokedR'][0][trial] > 0 :
        rat_choice = 1 # "R"
    else : 
        rat_choice = -1 # "L"
        
    return ratdata['rawdata']['rightbups'][0][trial][0], \
ratdata['rawdata']['leftbups'][0][trial][0], \
ratdata['rawdata']['T'][0][trial], rat_choice

In [3]:
# trial index starts from 0 
RightClickTimes, LeftClickTimes, maxT, rat_choice = trialdata(ratdata, 0) 
print LeftClickTimes
print rat_choice

[ 0.       0.19235  0.34361]
-1


In [4]:
import numpy as np

# Global variables 
epsilon = 10.0**(-10) 
dx = 0.25
dt = 0.02

# Parameters
sigma_a = 1; sigma_s = 0.1; sigma_i = 0.2; 
sigma_a_sbin = sigma_a  # remember we need this copy for Fmatrix
lam = -0.0005; B = 4.1; bias = 0.1; 
phi = 0.3; tau_phi = 0.1; lapse = 0.05;
params = [sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse]   

## bin_centers = make_bins(B, dx, binN)

In [5]:
def make_bins(B, dx, binN):
    '''
    inputs:
    -------
    - B is the value of the accumulator's sticky bound
    - dx is the spacing between all bins, except the first and last 
    - binN is the number of bins with bin centers > 0
    
    outputs:
    --------
    - bin_centers array of bin_centers
    '''
    bins = (np.arange(binN)+1)*B
    bins = dx*bins/B
    
    if bins[-1] == B:
        bins[-1] = B+dx
    else:
        bins[-1] = 2*B - bins[-2]
    
    bins = np.hstack((-bins[::-1], np.array([0]), bins))
    return bins

## F = Fmatrix([sigma, lambda, c], bin_centers)

In [6]:
def Fmatrix(params, bin_centers):
    global dt,dx
    sigma2 = params[0]
    lam = params[1]
    c = params[2]
    
    sigma2_sbin = sigma2
    
    F = np.zeros((bin_centers.size,bin_centers.size))
    
    if lam == 0:  # for Fi
        mus = bin_centers*np.exp(lam*dt)
    else:        
        mus = np.exp(lam*dt)*(bin_centers + c/lam) - c/lam

        
# modified it according to gaussbins.m -amyoon
#     sbinsize = 0.1*sigma_sbin
#     swidth = 4*sigma_sbin
#     sbins = np.arange(-swidth,swidth+epsilon,sbinsize)
    n_sbins = max(70, np.ceil(10*np.sqrt(sigma2_sbin)/dx))

    swidth = 5*np.sqrt(sigma2_sbin)
    sbins = np.linspace(-swidth,swidth,n_sbins*2+1)
    sbinsize = sbins[1]-sbins[0]

    ps = np.exp(-sbins**2/(2*sigma2)) 
    ps = ps/sum(ps)
            
    base_sbins = sbins
        
    for j in np.arange(bin_centers.size-1)+1:
        sbins = base_sbins + mus[j]
        tmp = 1
        
        for k in range(sbins.size):
            if sbins[k] <= bin_centers[0]:
                F[0,j] += ps[k]
            elif sbins[k] >= bin_centers[-1]:
                F[-1,j] += ps[k]
            else:
                if (sbins[k] > bin_centers[0] and sbins[k] < bin_centers[1]):
                    bottom = 0; top = 1;
                elif (sbins[k] > bin_centers[-2] and sbins[k] < bin_centers[-1]):
                    bottom = bin_centers.size-2; top = bin_centers.size-1;
                else :    
                    bottom = int(np.floor((sbins[k]-bin_centers[1])/dx) + 1);
                    top = int(np.ceil((sbins[k]-bin_centers[1])/dx) + 1);

                if bottom < 0: bottom = 0; 
                if top < 0: top = 0; 

                if bottom == top:
                    F[bottom,j] += ps[k]
                else:
                    F[top,j] += ps[k]*(sbins[k] - bin_centers[bottom])/(bin_centers[top] - bin_centers[bottom])
                    F[bottom,j] += ps[k]*(bin_centers[top] - sbins[k])/(bin_centers[top] - bin_centers[bottom])
                        
    F[:,0] = 0
    F[:,-1] = 0
    F[0,0] = 1
    F[-1,-1] = 1
    
    return F

#### LL = logProbRight(params)

params = [sigma_a, sigma_s, sigma_i, lambda, B, bias, phi, tau_phi, lapse]

In [12]:
import tensorflow as tf
import numpy as np


# =========================== Model ================================ #

def logLike(sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse, RightClickTimes, LeftClickTimes, maxT, rat_choice):
    tf_params = [sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse]
    if rat_choice > 0: # == "R":
        LP_right = logProbRight(tf_params, RightClickTimes, LeftClickTimes, maxT)
#         print "R : " + str(LP_right)
        return LP_right
    elif rat_choice < 0: # == "L":
        LP_right = logProbRight(tf_params, RightClickTimes, LeftClickTimes, maxT)
#         print "L : " + str(np.log(1 - np.exp(LP_right)))
        return np.log(1 - np.exp(LP_right))
    else:
        print -1. # "Rat did what?? It was neither R nor L"
        
def logProbRight(params, RightClickTimes, LeftClickTimes, maxT):
#     sigma_a = params['sigma_a']
#     sigma_s = params['sigma_s']
#     sigma_i = params['sigma_i']
#     lam = params['lambda']
#     B = params['B']
#     bias = params['bias']
#     phi = params['phi']
#     tau_phi = params['tau_phi']
#     lapse = params['lapse']
#     params = [sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse]  
    
    sigma2_a = params[0]
    sigma2_s = params[1]
    sigma2_i = params[2]
    lam = params[3]
    B = params[4]
    bias = params[5]
    phi = params[6]
    tau_phi = params[7]
    lapse = params[8]
    
    global a, dx, dt, a_trace, c_trace
        
    Nsteps = int(np.ceil(maxT/dt))

    if RightClickTimes.size == 0:
        RightClickTimes = np.empty(0)
    if LeftClickTimes.size == 0:
        LeftClickTimes = np.empty(0)
    
    net_input = np.zeros((1, Nsteps))
    total_input = np.zeros((1, Nsteps))

    NClicks = np.zeros((1, Nsteps))    
    
        
    # ====== Make adapted clicks
    
    Lsame = np.ones((1,np.size(LeftClickTimes)))
    Rsame = np.ones((1,np.size(RightClickTimes)))
    # magnitude of stereo clicks set to zero   
    Lsame[0] = 0; Rsame[0] = 0;
    
    # inter-click-intervals
    ici_L = np.diff(LeftClickTimes)
    ici_R = np.diff(RightClickTimes)
    
    for i in np.arange(np.size(LeftClickTimes)-1)+1:
        last_L = tau_phi*np.log(1-Lsame[0][i-1]*phi)
        Lsame[0][i] = 1 - np.exp((-ici_L[i-1] + last_L)/tau_phi)
        
    for i in np.arange(np.size(RightClickTimes)-1)+1:
        last_R = tau_phi*np.log(1-Rsame[0][i-1]*phi)
        Rsame[0][i] = 1 - np.exp((-ici_R[i-1] + last_R)/tau_phi)

    Lsame = Lsame.real    
    Rsame = Rsame.real   
        
    # index starts from 0
    # net_input / total_input
    # Counting number of clicks
    cnt = 0
    for i in np.ceil((RightClickTimes+epsilon)/dt).astype(int):
        NClicks[0][i-1] += 1
        net_input[0][i-1] += Rsame[0][cnt]
        total_input[0][i-1] += Rsame[0][cnt]
        cnt += 1
    
    cnt = 0 
    for i in np.ceil((LeftClickTimes+epsilon)/dt).astype(int):
        NClicks[0][i-1] += 1  
        net_input[0][i-1] -= Lsame[0][cnt]
        total_input[0][i-1] += Lsame[0][cnt]
        cnt += 1        
       
    binN = int(np.ceil(B/dx))
    
    bin_centers = make_bins(B,dx,binN)
        
    a_trace = np.zeros((bin_centers.size, Nsteps))
    c_trace = np.zeros((1, Nsteps+1))
    a0 = np.zeros(bin_centers.size)
    a0[binN] = 1-2*lapse; a0[0] = lapse; a0[-1] = lapse;

    
    Fi = Fmatrix([sigma2_i, 0, 0.0], bin_centers)
    
    a = np.matmul(Fi,a0)
    a_trace[:,0] = a.flatten()
 
    F0 = Fmatrix([sigma2_a*dt, lam, 0.0], bin_centers)
    for i in np.arange(Nsteps-1)+1:
        if total_input[0][i-1]==0:
            a = np.matmul(F0,a)
        else:
            total_var = sigma2_a*dt + (sigma2_s*total_input[0][i-1])/40
            F = Fmatrix([total_var, lam, net_input[0][i-1]/dt], bin_centers)
            a = np.matmul(F,a)
            
        c_trace[0][i] = net_input[0][i-1]
        a_trace[:,i] = a.flatten()

    bias_bottom = int(np.floor((bias-bin_centers[1])/dx) + 1)
    bias_top = int(np.ceil((bias-bin_centers[1])/dx) + 1) # top
    
    Pd = np.zeros((1, bin_centers.size))
    Pd[0][-bias_top+1:] = a[-bias_top+1:]
    if bias_bottom == bias_top:
        Pd[0][bias_bottom] = a[bias_bottom]/2
    else:
        dh = bin_centers[bias_top] - bias
        dl = bias - bin_centers[bias_bottom]
        dd = dh + dl
        Pd[0][bias_top] = a[bias_top]*(0.5 + dh/dd/2)
        Pd[0][bias_bottom] = a[bias_bottom]*(dh/dd/2)
    
    pright = sum(Pd[0])
    
    return np.log(pright)


# =========== tf Graph Input =========== #

## Variables 
# 1. (can assign the name of each variable later..)
# tf_params = tf.Variable(params, name = "params")

# 2. 
# tf_params = {
#     'sigma_a' : tf.Variable(sigma_a, name="sigma_a"),
#     'sigma_s' : tf.Variable(sigma_s, name="sigma_s"),
#     'sigma_i' : tf.Variable(sigma_i, name="sigma_i"),
#     'lamda' : tf.Variable(lam, name="lambda"),
#     'B' : tf.Variable(B, name="B"),
#     'bias' : tf.Variable(bias, name="bias"),
#     'phi' : tf.Variable(phi, name="phi"),
#     'tau_phi' : tf.Variable(tau_phi, name="tau_phi"),
#     'lapse' : tf.Variable(lapse, name="lapse")
# }

# 3.
# Parameters
sigma_a = 1; sigma_s = 0.1; sigma_i = 0.2; 
sigma_a_sbin = sigma_a  # remember we need this copy for Fmatrix
lam = -0.0005; B = 4.1; bias = 0.1; 
phi = 0.3; tau_phi = 0.1; lapse = 0.05;

sigma_a = tf.Variable(sigma_a, name="sigma_a")
sigma_s = tf.Variable(sigma_s, name="sigma_s")
sigma_i = tf.Variable(sigma_i, name="sigma_i")
lam = tf.Variable(lam, name="lambda")
B = tf.Variable(B, name="B")
bias = tf.Variable(bias, name="bias")
phi = tf.Variable(phi, name="phi")
tau_phi = tf.Variable(tau_phi, name="tau_phi")
lapse = tf.Variable(lapse, name="lapse")
# tf_params = [sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse]   

# ----------tip
# values = [1.0, 3.0]
# vars_ = [tf.Variable([v], dtype=dtype) for v in values]


## inputs
RightClickTimes = tf.placeholder(tf.float32, name = 'RightClickTimes') # Right Clicks
LeftClickTimes = tf.placeholder(tf.float32, name = 'LeftClickTimes') # Left Clicks
maxT = tf.placeholder(tf.float32, name = 'maxT')
rat_choice = tf.placeholder(tf.float32, name = 'rat_choice')


# ==================== Construct Model ========================= #

LL = tf.py_func(logLike, [sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse, 
                          RightClickTimes, LeftClickTimes, maxT, rat_choice], [tf.float64])

opt = tf.train.AdamOptimizer(0.1)
grads_and_vars = opt.compute_gradients(tf.to_float(LL[0]),[sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse])
grads2 = [g for g, _ in grads_and_vars]
vars2 = [v for _, v in grads_and_vars]

var_grad = tf.gradients(LL[0],[sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse])[0]

# ==================== Launch the Graph ========================= #

init = tf.initialize_all_variables()
with tf.Session() as sess:
    sess.run(init)
    
    # Set logs writer into folder /tmp/tensorflow_logs
    summary_writer = tf.train.SummaryWriter('/tmp/tensorflow_logs', graph_def = sess.graph_def)

    total_LL = 0.
    
    # Training cycle
    for i in range(27):
        RClickTimes1, LClickTimes1, maxT1, rat_choice1 = trialdata(ratdata, i)
        
        # Fit training using batch data
        LLi = sess.run(LL, feed_dict={RightClickTimes:RClickTimes1, 
                                      LeftClickTimes: LClickTimes1, 
                                      maxT:maxT1, 
                                      rat_choice : rat_choice1})[0]
        
#         var_grad_val = sess.run(var_grad, feed_dict={RightClickTimes:RClickTimes1, 
#                                             LeftClickTimes: LClickTimes1, 
#                                             maxT:maxT1, 
#                                             rat_choice : rat_choice1})

#         var_grad_val = sess.run(grads, feed_dict={RightClickTimes:RClickTimes1, 
#                                             LeftClickTimes: LClickTimes1, 
#                                             maxT:maxT1, 
#                                             rat_choice : rat_choice1})
#         grad_vals = sess.run(grads2)


        total_LL += LLi
        
        
        print "Trial :", '%04d' % (i+1), "LL = ", "{:.9f}".format(LLi)
    
    print "Done."


Trial : 0001 LL =  -2.535253892
Trial : 0002 LL =  -0.240068729
Trial : 0003 LL =  -0.053111613
Trial : 0004 LL =  -2.835791595
Trial : 0005 LL =  -0.186951994
Trial : 0006 LL =  -1.200522806
Trial : 0007 LL =  -0.051297237
Trial : 0008 LL =  -0.051327458
Trial : 0009 LL =  -1.489630181
Trial : 0010 LL =  -0.071373653
Trial : 0011 LL =  -0.979718816
Trial : 0012 LL =  -0.051501203
Trial : 0013 LL =  -0.244441988
Trial : 0014 LL =  -2.949191683
Trial : 0015 LL =  -0.542772727
Trial : 0016 LL =  -0.051361191
Trial : 0017 LL =  -0.053230288
Trial : 0018 LL =  -1.016906091
Trial : 0019 LL =  -0.052075662
Trial : 0020 LL =  -0.162860514
Trial : 0021 LL =  -0.053274167
Trial : 0022 LL =  -2.995303232
Trial : 0023 LL =  -0.797323782
Trial : 0024 LL =  -1.394077035
Trial : 0025 LL =  -0.067127882
Trial : 0026 LL =  -0.051295122
Trial : 0027 LL =  -0.094515498
Done.


In [10]:
grads_and_vars

[(None, <tensorflow.python.ops.variables.Variable at 0x117b44590>),
 (None, <tensorflow.python.ops.variables.Variable at 0x117455590>),
 (None, <tensorflow.python.ops.variables.Variable at 0x117455450>),
 (None, <tensorflow.python.ops.variables.Variable at 0x117b44610>),
 (None, <tensorflow.python.ops.variables.Variable at 0x117b60bd0>),
 (None, <tensorflow.python.ops.variables.Variable at 0x117b60b10>),
 (None, <tensorflow.python.ops.variables.Variable at 0x117bc2790>),
 (None, <tensorflow.python.ops.variables.Variable at 0x117b72d50>),
 (None, <tensorflow.python.ops.variables.Variable at 0x117455850>)]

In [ ]:
# interactive debugging
sess = tf.InteractiveSession()

init = tf.initialize_all_variables()

sess.run(init)

# Btmp = log_likelihood(tf_params, "R", RightPulseTimes=np.array([0.2, 0.4]))
# bb = tf.equal(rat_choice.eval(),'R')
Btmp = tf_params
sess.run(Btmp)
print(Btmp.eval())

sess.close()

In [11]:

LL = logLike(params,rat_choice)
optimizer = tf.train.AdamOptimizer(learning_rate=0.1).minimize(LL, var_list=tf_params)

TypeError: logLike() takes exactly 13 arguments (2 given)

In [ ]:
# tf_params = tf.convert_to_tensor(params)
# tf_params = params
tf_params = tf.Variable(params)

# Construct model
LL = tf.py_func(logLike, [tf_params], [tf.float32])


In [ ]:
tf_params

In [ ]:
tf_params = tf.Variable(params)

init = tf.initialize_all_variables()

loss = logLike(tf_params)
opt = tf.train.AdamOptimizer(0.1)
grads = opt.compute_gradients(loss)

grad_placeholder = [tf.float32]
apply_placeholder_op = opt.apply_gradients(grad_placeholder)
transform_grads = [(function1(grad[0]), grad[1]) for grad in grads]
apply_transform_op = opt.apply_gradients(transform_grads)

with tf.Session() as session:
    session.run(init)
    RClickTimes, LClickTimes, maxT, rat_choice = trialdata(ratdata, 10)
    tt = session.run(grads)
    print(tt)


In [ ]:
init = tf.initialize_all_variables()

with tf.Session() as session:
    session.run(init)
    global a, dx, dt, Nsteps, LeftClickTimes, RightClickTimes, a_trace, c_trace, bin_centers

    RClickTimes, LClickTimes, maxT, rat_choice = trialdata(ratdata, 10)
#     X = session.run(LL, feed_dict={tf_params: params})
    tt = session.run(LL, feed_dict={tf_params: tf_params})[0]
    print(tt)

In [ ]:
# Adam Optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(LL) 

In [ ]:
# Initializing the variables
init = tf.initialize_all_variables()

In [ ]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    # Training cycle
    for epoch in range(15):
        avg_LL = 0.
        # Loop over trials

        for i in range(10):
            RClickTimes, LClickTimes, maxT, rat_choice = trialdata(ratdata, i)

            # Fit training using batch data
            sess.run(LL, feed_dict={Rc:RClickTimes, Lc: LClickTimes, T:maxT, rat_choice:rat_choice})
            # Compute average loss
            avg_LL += sess.run(LL, feed_dict={x:batch_xs,y:batch_ys})/total_batch
            
        # Display logs per epoch step
        if epoch % display_step ==0:
            print "Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost)
    print "Optimization Done."
    
    # Test model
    correct_prediction = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))
    print "Accuracy:", accuracy.eval({x:mnist.test.images, y: mnist.test.labels})

In [ ]:
# TensorFlow Variables

tf_params = {
    'sigma_a' : tf.Variable(sigma_a, name="sigma_a"),
    'sigma_s' : tf.Variable(sigma_s, name="sigma_s"),
    'sigma_i' : tf.Variable(sigma_i, name="sigma_i"),
    'lambda' : tf.Variable(lam, name="lambda"),
    'B' : tf.Variable(B, name="B"),
    'bias' : tf.Variable(bias, name="bias"),
    'phi' : tf.Variable(phi, name="phi"),
    'tau_phi' : tf.Variable(tau_phi, name="tau_phi"),
    'lapse' : tf.Variable(lapse, name="lapse")
}

In [ ]:
B = 2.3
dx = 1
binN = np.ceil(B/dx)

bin_centers = make_bins(B,dx,binN)
bin_centers

In [ ]:
F = Fmatrix(params,bin_centers)
F

In [ ]:
def logProbRight(params):
    print(params['sigma_a'].eval())
    sigma_a = params['sigma_a'].eval()
    sigma_s = params['sigma_s'].eval()
    sigma_i = params['sigma_i'].eval()
    lam = params['lambda'].eval()
    B = params['B'].eval()
    bias = params['bias'].eval()
    phi = params['phi'].eval()
    tau_phi = params['tau_phi'].eval()
    lapse = params['lapse'].eval()
    
    global a, dx, dt, Nsteps, LeftClickTimes, RightClickTimes, a_trace, c_trace, bin_centers
        
    LeftClicks = np.zeros((1, Nsteps))
    RightClicks = np.zeros((1, Nsteps))

    if RightClickTimes.size == 0:
        RightClickTimes = np.empty(0)
    if LeftClickTimes.size == 0:
        LeftClickTimes = np.empty(0)
    
    # index starts from 0
    for i in np.ceil((RightClickTimes+epsilon)/dt).astype(int):
        RightClicks[0][i-1] = 1
    for i in np.ceil((LeftClickTimes+epsilon)/dt).astype(int):
        LeftClicks[0][i-1] = 1   

    binN = int(np.ceil(B/dx))
    binBias = int(np.floor(bias/dx)) + binN + 1
    bin_centers = make_bins(B,dx,binN)
    
    a_trace = np.zeros((bin_centers.size, Nsteps+1))
    c_trace = np.zeros((1, Nsteps+1))
    a = np.zeros(bin_centers.size)#*sigma_a*0.0; 
    a[binN] = 1-2*lapse; a[0] = lapse; a[-1] = lapse;
    c_eff   = 1
    
    Fi = Fmatrix([sigma_i/np.sqrt(dt), lam, 0.0], bin_centers)
    a = np.matmul(Fi,a)
    a_trace[:,0] = a.flatten()
    
    F0 = Fmatrix([sigma_a, lam, 0.0], bin_centers)
    for i in range(Nsteps):
        c_eff = 1 + (c_eff - 1)*np.exp(-dt/tau_phi)
        if (RightClicks[0][i]==0) & (LeftClicks[0][i]==0):
            a = np.matmul(F0,a)
        elif (RightClicks[0][i]==1) & (LeftClicks[0][i]==1):
            c_eff = 0
            a = np.matmul(F0,a)
        else:
            net_sigma = np.sqrt(sigma_a**2 + (sigma_s*c_eff)**2/dt)
            F = Fmatrix([net_sigma, lam, (RightClicks[0][i] - LeftClicks[0][i])*c_eff/dt], bin_centers)
            a = np.matmul(F,a)
            c_eff = c_eff*phi
        
        c_trace[0][i+1] = c_eff
        a_trace[:,i+1] = a.flatten()

    pright = sum(a[-binBias+1:]) + a[binBias-1]*0.5*(dx/2 - (bias - bin_centers[binBias-1]))/(dx/2)

    return np.log(pright)

In [ ]:
import matplotlib.pylab as plt
%matplotlib inline

LL = logProbRight(params)

im = plt.imshow(a_trace)
plt.colorbar()
im.set_extent([0,maxT*20,bin_centers[0], bin_centers[-1]])
plt.show()

plt.plot(bin_centers,a)

In [ ]:
def logLike(params):
    if rat_choice == "R":
        LP_right = logProbRight(params)
        return LP_right, a_trace
    elif rat_choice == "L":
        LP_right = logProbRight(params)
        return np.log(1 - np.exp(LP_right))
    else:
        print "Rat did what?? It was neither R nor L"

In [ ]:
logLike(params)

In [ ]:
def llikey(params, rat_choice, maxT=1, RightPulseTimes=np.array([]), LeftPulseTimes=np.array([]), dx=0.25, dt=0.02):

    global RightClickTimes, LeftClickTimes, Nsteps
    
    RightClickTimes = RightPulseTimes.compress((RightPulseTimes <= maxT).flat)
    LeftClickTimes  = LeftPulseTimes.compress((LeftPulseTimes <= maxT).flat)
    Nsteps = int(np.ceil(maxT/dt))
    
    B = params[4].eval()
    binN = int(np.ceil(B/dx))
    bin_centers = make_bins(B, dx, binN); 
    bin_times = dt*np.arange(Nsteps+1);
    
    LL = logLike(params)
#     LLgrad = tf.gradients(LL, params)[0]
#     hess = tf.map_fn(lambda grid: tf.gradients(grad,))
    
    return LL
#     return LL,LLgrad
#     return LL,LLgrad, LLhessian, length(params) # 

In [ ]:
### =============== testing 1 ================= ####

# Global variables 
epsilon = 10.0**(-10) 
dx = 0.125 
dt = 0.02
Nsteps = int(np.ceil(1.0/dt))

# Parameters
sigma_a = 1; sigma_s = 0.1; sigma_i = 0.2; 
sigma_a_sbin = sigma_a  # remember we need this copy for Fmatrix
lam = -0.0005; B = 4.1; bias = 0.1; 
phi = 0.3; tau_phi = 0.1; lapse = 0.05;
params = [sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse]   

# trial index starts from 0 
RightClickTimes, LeftClickTimes, maxT, rat_choice = trialdata(ratdata, 0) 

logLike(params)

### =========================================== ####

In [ ]:
### =============== testing 2 ================= ####

# Global variables 
epsilon = 10.0**(-10) 
dx = 0.125 
dt = 0.02
Nsteps = int(np.ceil(1.0/dt))

# Parameters
sigma_a = 1; sigma_s = 0.1; sigma_i = 0.2; 
sigma_a_sbin = sigma_a  # remember we need this copy for Fmatrix
lam = -0.0005; B = 4.1; bias = 0.1; 
phi = 0.3; tau_phi = 0.1; lapse = 0.05;
params = [sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse]   

LL = llikey(params, "R", RightPulseTimes=np.array([0.2, 0.4]))
LL
### =========================================== ####

In [ ]:
%matplotlib inline
RightClickTimes, LeftClickTimes, maxT, rat_choice = trialdata(ratdata, 9) 

LL = logProbRight(params)

plt.imshow(a_trace)
plt.colorbar()
plt.show()

np.shape(bin_centers)

In [ ]:
def multiLL(ratdata, params, ntrials):
    LL = 0
    LLgrad = np.zeros((np.shape(params)[0]))
    LLhessian = np.zeros((np.shape(params)[0],np.shape(params)[0]))
    
    for i in range(ntrials):
        RClickTimes, LClickTimes, maxT, rat_choice = trialdata(ratdata, i)
#         LLi, LLgradi, LLhessiani = llikey(params, rat_choice, maxT=maxT, RightPulseTimes=RClickTimes,
        LLi = llikey(params, rat_choice, maxT=maxT, RightPulseTimes=RClickTimes, LeftPulseTimes = LClickTimes, dx=0.25, dt = 0.02)

        LL        = LL + LLi;
#         LLgrad    = LLgrad + LLgradi;
#         LLhessian = LLhessian + LLhessiani;

        print(str(i+1)+ " - LL : " + str(LL))
        
    return LL, LLgrad, LLhessian

In [ ]:
epsilon = 10.0**(-10); dx = 0.125; dt = 0.02; Nsteps = int(np.ceil(1.0/dt))

sigma_a = 1; sigma_s = 0.1; sigma_i = 0.2; 
sigma_a_sbin = sigma_a  # remember we need this copy for Fmatrix
lam = -0.0005; B = 4.1; bias = 0.1; 
phi = 0.3; tau_phi = 0.1; lapse = 0.05;
params = [sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse]   

multiLL(ratdata, params, 10)

In [ ]:
sess.close()

## Tensorflow..

In [ ]:
tf_params = tf.Variable(params, name = "params")

In [ ]:
import tensorflow as tf

# Parameters
sigma_a = 1; sigma_s = 0.1; sigma_i = 0.2; 
sigma_a_sbin = sigma_a  # remember we need this copy for Fmatrix
lam = -0.0005; B = 4.1; bias = 0.1; 
phi = 0.3; tau_phi = 0.1; lapse = 0.05;
params = [sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse]   

# tf_params = tf.Variable(params, name = "params")
tf_params = {
    'sigma_a' : tf.Variable(sigma_a, name="sigma_a"),
    'sigma_s' : tf.Variable(sigma_s, name="sigma_s"),
    'sigma_i' : tf.Variable(sigma_i, name="sigma_i"),
    'lambda' : tf.Variable(lam, name="lambda"),
    'B' : tf.Variable(B, name="B"),
    'bias' : tf.Variable(bias, name="bias"),
    'phi' : tf.Variable(phi, name="phi"),
    'tau_phi' : tf.Variable(tau_phi, name="tau_phi"),
    'lapse' : tf.Variable(lapse, name="lapse")
}

loss = logLike(tf_params)

opt = tf.train.AdamOptimizer(0.1)
grads = opt.compute_gradients(loss)

grad_placeholder = [tf.float32]
apply_placeholder_op = opt.apply_gradients(grad_placeholder)
transform_grads = [(function1(grad[0]), grad[1]) for grad in grads]
apply_transform_op = opt.apply_gradients(transform_grads)

# Initialize
sess = tf.InteractiveSession()

init = tf.initialize_all_variables()

sess.run(init)

grad_vals = sess.run(grads)

In [ ]:
tf_params

In [ ]:
mat1 = tf.constant([[3., 3.]])
mat2 = tf.constant([[2.],[2.]])

product = tf.matmul(mat1,mat2)

In [ ]:
sess = tf.Session()
result = sess.run(product)
print result
sess.close()

In [ ]:
with tf.Session() as sess:
    result = sess.run([product])
    print(result)

In [ ]:
sess = tf.InteractiveSession()

x = tf.Variable([1.0, 2.0])
a = tf.constant([3.0, 3.0])

x.initializer.run()

sub = tf.sub(x, a)
print(sub.eval())

sess.close()

In [ ]:
state = tf.Variable(0, name="counter")

one = tf.constant(1)
new_value = tf.add(state, one)
update = tf.assign(state, new_value)

init_op = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init_op)
    
#     print(sess.run(state))
    
    for _ in range(3):
        sess.run(update)
        print(sess.run(state))

In [ ]:
input1 = tf.constant(3.0)
input2 = tf.constant(2.0)
input3 = tf.constant(5.0)
intermed = tf.add(input2, input3)
mul = tf.mul(input1, intermed)

with tf.Session() as sess:
    result = sess.run([mul, intermed])
    print(result)
    


In [ ]:
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.mul(input1, input2)

with tf.Session() as sess:
    print(sess.run([output], feed_dict={input1:[7.], input2:[2.]}))

In [ ]:
import tensorflow as tf
from matplotlib import pyplot as plt

shape = (50, 50)
initial_board = tf.random_uniform(shape, minval=0, maxval=2, dtype=tf.int32)

with tf.Session() as session:
    X = session.run(initial_board)

fig = plt.figure()
plot = plt.imshow(X, cmap='Greys',  interpolation='nearest')

In [ ]:
import numpy as np
from scipy.signal import convolve2d

def update_board(X):
    # Check out the details at: https://jakevdp.github.io/blog/2013/08/07/conways-game-of-life/
    # Compute number of neighbours,
    N = convolve2d(X, np.ones((3, 3)), mode='same', boundary='wrap') - X
    # Apply rules of the game
    X = (N == 3) | (X & (N == 2))
    return X

In [ ]:
board = tf.placeholder(tf.int32, shape=shape, name='board')
board_update = tf.py_func(update_board, [board], [tf.int32])

In [ ]:
with tf.Session() as session:
    initial_board_values = session.run(initial_board)
    X = session.run(board_update, feed_dict={board: initial_board_values})[0]

In [ ]:
import matplotlib.animation as animation

def game_of_life(*args):
    X = session.run(board_update, feed_dict={board: X})[0]
    plot.set_array(X)
    return plot,

fig = plt.figure()
ani = animation.FuncAnimation(fig, game_of_life, interval=200, blit=True)
plt.show()

In [ ]:
import tensorflow as tf
import numpy as np

# Create 100 phony x, y data points in NumPy, y = x * 0.1 + 0.3
x_data = np.random.rand(100).astype(np.float32)
y_data = x_data * 0.1 + 0.3

# Try to find values for W and b that compute y_data = W * x_data + b
# (We know that W should be 0.1 and b 0.3, but Tensorflow will
# figure that out for us.)
W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.zeros([1]))
y = W * x_data + b

# Minimize the mean squared errors.
loss = tf.reduce_mean(tf.square(y - y_data))
optimizer = tf.train.GradientDescentOptimizer(0.5)

grads_and_vars = optimizer.compute_gradients(loss,[W, b])
grads2 = [g for g, _ in grads_and_vars]
vars2 = [v for _, v in grads_and_vars]

train = optimizer.minimize(loss)

# Before starting, initialize the variables.  We will 'run' this first.
init = tf.initialize_all_variables()

# Launch the graph.
sess = tf.Session()
sess.run(init)

# Fit the line.
for step in xrange(201):
    sess.run(train)
    if step % 20 == 0:
        print(step, sess.run(W), sess.run(b))
        print sess.run(grads2)

# Learns best fit is W: [0.1], b: [0.3]

In [ ]:
grads2

In [ ]:
vars2

In [ ]:
grads